In [12]:
import pandas as pd

from utils.common_transformers import DateHandler, IsNull, Dropper, DTypeTransformer, FeatureSelector

from sklearn.preprocessing import StandardScaler

from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline

from sklearn.linear_model import LinearRegression, Lasso, Ridge, BayesianRidge
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_squared_log_error, make_scorer

import numpy as np

In [13]:
train = pd.read_csv("../data/Train.csv")
train.dtypes

session_id         object
session_number      int64
client_agent       object
device_details     object
date               object
purchased           int64
added_in_cart       int64
checked_out         int64
time_spent        float64
dtype: object

In [14]:
l = ["a","b","c"]
l1 = l
l1.remove("a")
l2 = l.copy()
l2.remove("c")
print(l)

['b', 'c']


In [15]:
data_prepping = make_pipeline(
    FeatureSelector(["client_agent","date","purchased","added_in_cart","checked_out"]),
    DateHandler(["date"],date_format="%Y-%m-%d",include=["quater","is_holiday","close_to_month_start_end"]),
    IsNull(include=["client_agent"]),
    StandardScaler()
)

In [16]:
models = {
    "LR" : LinearRegression(),
    "Lasso" : Lasso(alpha=0.8),
    "Ridge" : BayesianRidge(),
    "RFR" : RandomForestRegressor(criterion="mse"),
    "ABR" : AdaBoostRegressor(base_estimator=LinearRegression())
}

scores = {
    
}

In [17]:
X = train.drop("time_spent",axis=1)
y = train["time_spent"]
X_train, X_test, y_train, y_test = train_test_split(train.drop("time_spent",axis=1),train["time_spent"])

In [18]:
def calculate_root_mean_squared_log_error(y_true, y_pred):
    """Calculate root mean squared error of log(y_true) and log(y_pred)"""
    if len(y_pred)!=len(y_true): return 'error_mismatch'
    try:
        y_pred = np.where(y_pred<0,0.1,y_pred)
        y_pred_new = [math.log(x) for x in y_pred]
        y_true_new = [math.log(x) for x in y_true]
        return mean_squared_log_error(y_true_new, y_pred_new)
    except Exception:
        print("Problem in scoring logic")

In [29]:
scorer = make_scorer(calculate_root_mean_squared_log_error)
pipe = make_pipeline(data_prepping,Lasso())
cross_val_score(pipe,X,y,cv=5,scoring=scorer)

array([nan, nan, nan, nan, nan])

In [30]:
pipe.fit(X,y)
y_pred = pipe.predict(X_test)
y_pred = np.where(y_pred<0,0,y_pred)

In [31]:
mean_squared_log_error(y_test,y_pred)

3.9886820656848876

In [31]:
pipe.fit(X,y)
pipe.predict(X)

['client_agent', 'purchased', 'added_in_cart', 'checked_out', 'date_close_to_month_start_end', 'date_is_holiday', 'date_quater', 'date_close_to_month_start_end', 'date_is_holiday', 'date_quater']
['client_agent', 'purchased', 'added_in_cart', 'checked_out', 'date_close_to_month_start_end', 'date_is_holiday', 'date_quater', 'date_close_to_month_start_end', 'date_is_holiday', 'date_quater']


array([1227.263253  , 1097.63291664,  312.15468778, ...,  441.78502415,
        212.52001338,  464.30548732])